# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [100]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json


# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [101]:
# Store filepath in a variable
weather = "../weather_data.csv"

# Read our Data file with the pandas library
weather_data = pd.read_csv(weather)

#Convert everything to float values
weather_data["Max Temp"] = pd.to_numeric(weather_data["Max Temp"])

# Show just the header
weather_data.head(10)


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Fallon,90,US,1584227438,18,39.47,-118.78,62.60,31.09
1,Butaritari,88,KI,1584227737,72,3.07,172.79,83.62,10.76
2,Rikitea,0,PF,1584227737,78,-23.12,-134.97,78.26,12.12
3,Jamestown,90,US,1584227360,80,42.10,-79.24,37.99,6.93
4,Punta Arenas,75,CL,1584227219,67,-53.15,-70.92,57.20,16.11
5,Constitución,38,CL,1584227167,83,-35.33,-72.42,54.82,5.70
6,Katsuura,20,JP,1584227738,60,35.13,140.30,44.01,10.29
7,Isangel,40,VU,1584227739,74,-19.55,169.27,87.80,11.41
8,Ushuaia,40,AR,1584227634,62,-54.80,-68.30,53.60,6.93
9,Semey,36,KZ,1584227740,79,50.41,80.23,29.62,5.17


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [105]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = weather_data[["Lat", "Lng"]].astype(float)
humidity = weather_data["Humidity"].astype(float)

In [106]:
# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [109]:
# Narrow the DF to find other cities that meet "perfect" weather conditions.

ideal_weather = weather_data.loc[
    (weather_data["Max Temp"] > 70) & 
    (weather_data["Max Temp"] < 80) &
    (weather_data["Wind Speed"] < 10) &
    (weather_data["Cloudiness"] == 0)]

ideal_weather

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
68,Kaeo,0,NZ,1584227765,61,-35.10,173.78,75.00,5.01
122,Marawi,0,PH,1584227697,84,8.00,124.29,75.00,5.01
248,Kāvali,0,IN,1584227849,84,14.92,79.98,74.66,7.56
252,Araranguá,0,BR,1584227850,90,-28.93,-49.49,71.60,2.98
350,Kibala,0,TD,1584227894,29,9.11,18.35,78.80,4.61
374,Xai-Xai,0,MZ,1584227903,96,-25.05,33.64,70.88,1.81
421,Taoudenni,0,ML,1584227720,12,22.68,-3.98,70.63,2.93
426,Werda,0,BW,1584227923,40,-25.27,23.28,71.26,3.49
649,Inhambane,0,MZ,1584228088,82,-23.86,35.38,77.63,6.55


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [108]:
hotel_df = ideal_weather

hotel_df["Hotel Name"] = ""

hotel_df

/Users/jason/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
68,Kaeo,0,NZ,1584227765,61,-35.10,173.78,75.00,5.01,
122,Marawi,0,PH,1584227697,84,8.00,124.29,75.00,5.01,
248,Kāvali,0,IN,1584227849,84,14.92,79.98,74.66,7.56,
252,Araranguá,0,BR,1584227850,90,-28.93,-49.49,71.60,2.98,
350,Kibala,0,TD,1584227894,29,9.11,18.35,78.80,4.61,
374,Xai-Xai,0,MZ,1584227903,96,-25.05,33.64,70.88,1.81,
421,Taoudenni,0,ML,1584227720,12,22.68,-3.98,70.63,2.93,
426,Werda,0,BW,1584227923,40,-25.27,23.28,71.26,3.49,
649,Inhambane,0,MZ,1584228088,82,-23.86,35.38,77.63,6.55,


In [110]:
# Find the name of hotels near the city coordinates

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    response = name_address.json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [111]:
#Remote cities without hotel finds.

hotels = hotel_df[hotel_df["Hotel Name"] != ""]
hotels

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
122,Marawi,0,PH,1584227697,84,8.00,124.29,75.00,5.01,Marawi Resort Hotel
248,Kāvali,0,IN,1584227849,84,14.92,79.98,74.66,7.56,Hotel Mourya Paradise
252,Araranguá,0,BR,1584227850,90,-28.93,-49.49,71.60,2.98,Hotel Tourist Araranguá
350,Kibala,0,TD,1584227894,29,9.11,18.35,78.80,4.61,Hôtel des Chasses
649,Inhambane,0,MZ,1584228088,82,-23.86,35.38,77.63,6.55,Hotel Casa Do Capitao


In [112]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotels.iterrows()]
locations = hotels[["Lat", "Lng"]]

In [113]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))